<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/langgraph_human_interrupt_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental" "aiosqlite"

In [2]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')

# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

# Define a Simple Agent State;
# Agent State is accessible to all parts of the graph.
class AgentState(TypedDict):
  messages: Annotated[list[AnyMessage], operator.add]

In [5]:
tool = TavilySearchResults(max_results=2)
print(tool.name)

tavily_search_results_json


/tmp/ipython-input-983460408.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


In [6]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [7]:
# Define a simple agent
class Agent:
  def __init__(self, model, checkpointer, tools ,system=""):
    self.system = system
    graph = StateGraph(AgentState)
    graph.add_node("llm",self.call_llm)
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
        "llm",
        self.exists_action,
        {True: "action", False: END}
    )
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile(checkpointer = checkpointer)
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def exists_action(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0

  def call_llm(self, state: AgentState):
    messages = state["messages"]
    if self.system:
      messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    next_tools = message.tool_calls if message.tool_calls else ["END"]
    #print(f"Next tools: {next_tools}")
    return {'messages': [message]}

  def take_action(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
      print(f"Calling function: {t}")
      result = self.tools[t['name']].invoke(t['args'])
      print(f"Result from calling function: ", result)
      results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
    print("Back to the model")
    return {'messages': results}

In [8]:
prompt = """You are a smart research assistant. Use the search engine to query results. \
You are allowed to make multiple calls (either together or in sequence).\
Only look up information when you are sure of what you want. \
"""
abot = Agent(llm, memory, [tool], system=prompt)

In [9]:
# Define an graph with human interruption
class InterruptibleAgent(Agent):
    def __init__(self, model, checkpointer, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer, interrupt_before=["action"])
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

In [25]:
abot_itr = InterruptibleAgent(llm, memory, [tool], system=prompt)
thread = {"configurable": {"thread_id": "1"}}

messages = [HumanMessage(content="When is Chinese Spring Festival for 2026?")]
for event in abot_itr.graph.stream({"messages": messages}, thread):
  for v in event.values():
    print(v)

{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Chinese Spring Festival 2026 date"}'}, '__gemini_function_call_thought_signatures__': {'92ecdab8-ca34-4f7d-a9fb-391bd60e7eeb': 'CvABAb4+9vvSJVXFN2ULE3kvXoykv3ToOUMwBRUUVi8A/yMKQ4it1LuaUfbRLau+4/WYJk4/nzfKeJmnS/sqzhQZBlVscomj6+r0VsEDcrxnn/BWEyQ8TZQEFlbS+whuaES3js55CaTMlXCZvs7snUnQtgsHgcF95c7TRdiEdzKMnBV/dO7oiWugsdlflRFc65FL0gN4qAHhQyNh60WPj9gA8hWGrX5X12ol+uqsY6HQ+1pdLAuMOVEgvAWuDlfs5fpdbHVMSnueFQja8RZ8ceacxFiW0khChe1i7Rv93Eu21vLGeysQNqhzeW0/01fzJLON'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c473a-16cb-7ec3-a51c-79e6f8aa6373-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Chinese Spring Festival 2026 date'}, 'id': '92ecdab8-ca34-4f7d-a9fb-391bd60e7eeb', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'inp

In [26]:
while abot_itr.graph.get_state(thread).next:
  print(abot_itr.graph.get_state(thread))
  print(abot_itr.graph.get_state(thread).next)
  _input = input("proceed?")
  if _input != 'y':
    print("aborting!")
    break
  for event in abot_itr.graph.stream(None, thread):
    for v in event.values():
      print(v)

StateSnapshot(values={'messages': [HumanMessage(content='When is Chinese Spring Festival for 2026?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Chinese Spring Festival 2026 date"}'}, '__gemini_function_call_thought_signatures__': {'92ecdab8-ca34-4f7d-a9fb-391bd60e7eeb': 'CvABAb4+9vvSJVXFN2ULE3kvXoykv3ToOUMwBRUUVi8A/yMKQ4it1LuaUfbRLau+4/WYJk4/nzfKeJmnS/sqzhQZBlVscomj6+r0VsEDcrxnn/BWEyQ8TZQEFlbS+whuaES3js55CaTMlXCZvs7snUnQtgsHgcF95c7TRdiEdzKMnBV/dO7oiWugsdlflRFc65FL0gN4qAHhQyNh60WPj9gA8hWGrX5X12ol+uqsY6HQ+1pdLAuMOVEgvAWuDlfs5fpdbHVMSnueFQja8RZ8ceacxFiW0khChe1i7Rv93Eu21vLGeysQNqhzeW0/01fzJLON'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c473a-16cb-7ec3-a51c-79e6f8aa6373-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Chinese Spring Fe

In [27]:
states = []
for state in abot_itr.graph.get_state_history(thread):
  print(state)
  print("----")
  states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='When is Chinese Spring Festival for 2026?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Chinese Spring Festival 2026 date"}'}, '__gemini_function_call_thought_signatures__': {'92ecdab8-ca34-4f7d-a9fb-391bd60e7eeb': 'CvABAb4+9vvSJVXFN2ULE3kvXoykv3ToOUMwBRUUVi8A/yMKQ4it1LuaUfbRLau+4/WYJk4/nzfKeJmnS/sqzhQZBlVscomj6+r0VsEDcrxnn/BWEyQ8TZQEFlbS+whuaES3js55CaTMlXCZvs7snUnQtgsHgcF95c7TRdiEdzKMnBV/dO7oiWugsdlflRFc65FL0gN4qAHhQyNh60WPj9gA8hWGrX5X12ol+uqsY6HQ+1pdLAuMOVEgvAWuDlfs5fpdbHVMSnueFQja8RZ8ceacxFiW0khChe1i7Rv93Eu21vLGeysQNqhzeW0/01fzJLON'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c473a-16cb-7ec3-a51c-79e6f8aa6373-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Chinese Spring Fe

In [28]:
to_replay = states[-1]
to_replay

StateSnapshot(values={'messages': []}, next=('__start__',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f106706-09b5-6fc1-bfff-b6a3d3094ffa'}}, metadata={'source': 'input', 'step': -1, 'parents': {}}, created_at='2026-02-10T11:05:15.209504+00:00', parent_config=None, tasks=(PregelTask(id='f162a749-171b-44b9-34e1-a16f86429d1f', name='__start__', path=('__pregel_pull', '__start__'), error=None, interrupts=(), state=None, result={'messages': [HumanMessage(content='When is Chinese Spring Festival for 2026?', additional_kwargs={}, response_metadata={})]}),), interrupts=())

In [ ]:
# Pending: Update a tool message
#_id = to_replay.values["messages"][-1].tool_calls[0]['id']
#state_update = {"messages": [ToolMessage(
#    tool_call_id = _id,
#    name = "call_search_results_json",
#    content = "54 degree celcius",
#)]}
#branch_and_add = abot_itr.graph.update_state(
#    to_replay.config,
#    state_update,
#    as_node="action"
#)